In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import src.modules as module 
import math

In [3]:
torch.set_grad_enabled(False)

## Generate Dataset

In [4]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    
    target = abs(input.pow(2).sum(1).sub(1 / (2*math.pi)).sign().add(-1).div(2).long())
    return input, target

In [5]:
N = 5
train_input, train_target = generate_disc_set(N)
test_input, test_target = generate_disc_set(N)

In [6]:
#Testing mse
loss = module.LossMSE()
inputs = torch.tensor([[ 3., 0., 0., 0. ]])
targets = torch.tensor([[ 3., 0., 5., 0. ]])
loss(inputs, targets)



tensor(6.2500)

In [7]:
#testing sequential, tanh, linear, relu
model = module.Sequential(('lin1',module.Linear(4,3)),('tanh1',module.Tanh()),('tanh1',module.ReLU()))

print(model[1].name)
print(model(inputs))

tanh1
tensor([[0.6780, 0.9024, 0.7412]])
